# 02 · Silver — Clean eGlobal

| Item | Detail |
|------|--------|
| **Source** | `APAC_CRM_Analytics_LH.src_eglobal_premium_report` |
| **Target** | `APAC_Reporting_LH.clean_eglobal_chloe` |
| **Grain** | One row per invoice line |

### Alteryx Tool Mapping

| Cell | Alteryx Tool(s) | Description |
|------|----------------|-------------|
| 2 | Input (35,70,73,76,293,303), Filter (302,84) | Read Bronze, filter ACCIDENT (AU) & Employee Benefits |
| 3 | Formula (51,68,71,74,77,296), Formula (89,88,87,86,83,294), Join (304), Formula (204) | Date fallbacks, Revenue Country, CCY, branch mapping (HK), TRIM/UPPER |
| 5 | Join (61,198,199), Formula (66), Select (195) | Currency/Product/Insurer joins, USD calc, derived cols, final select |
| 6 | Output | Write Delta to Silver |

In [ ]:
# =============================================================================
# Cell 1: Setup & Configuration
# =============================================================================
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, DoubleType, DateType, IntegerType, LongType, FloatType, DecimalType
from pyspark.sql.utils import AnalysisException

# Lakehouse
BRONZE_LH = "APAC_CRM_Analytics_LH"
SILVER_LH = "APAC_Reporting_LH"

# Tables
SOURCE_TABLE = f"{BRONZE_LH}.src_eglobal_premium_report"
TARGET_TABLE = f"{SILVER_LH}.clean_eglobal_chloe"

# Reference tables
REF_BRANCH_MAPPING   = f"{BRONZE_LH}.ref_eglobal_premium_branch_mapping"
REF_CURRENCY_MAPPING = f"{BRONZE_LH}.ref_Chloe_asia_currency_mapping"
REF_PRODUCT_MAPPING  = f"{BRONZE_LH}.ref_Chloe_eglobal_product_mapping"
REF_INSURER_MAPPING  = f"{BRONZE_LH}.ref_Chloe_insurer_mapping"

In [ ]:
# =============================================================================
# Cell 2: Read Bronze + Filters
# Alteryx: Input tools (35,70,73,76,293,303) → Filter (302) ACCIDENT for AU
#          → Union (79) → Filter (84) Employee Benefits
# =============================================================================
df = spark.table(SOURCE_TABLE)
print(f"Source rows: {df.count()}")
print(f"Source columns: {len(df.columns)}")

# --- Filter ACCIDENT: only for Australia (Alteryx Tool 302) ---
# For AU rows, exclude records where LINE OF BUSINESS contains 'ACCIDENT'
# For non-AU rows, keep all records
df = df.filter(
    (~F.upper(F.col("`Source.Name`")).contains("AUSTRALIA")) |
    (F.col("`LINE OF BUSINESS`").isNull()) |
    (~F.upper(F.col("`LINE OF BUSINESS`")).contains("ACCIDENT"))
)

# --- Filter Employee Benefits (Alteryx Tool 84) — applies to ALL countries ---
df = df.filter(
    (F.col("`LINE OF BUSINESS`").isNull()) |
    (F.col("`LINE OF BUSINESS`") != "Employee Benefits")
)

print(f"After filters: {df.count()}")

In [ ]:
# =============================================================================
# Cell 3: Date Formulas, Revenue Country, CCY, Branch Mapping, TRIM/UPPER
# Alteryx: Formula tools per country (51,68,71,74,77,296)
#          Date formulas (89,88,87,86,83,294)
#          Branch mapping join (304) — HK only
#          TRIM/UPPER + null handling (204)
# =============================================================================

# --- INVOICE DATE NEXT YEAR ---
df = df.withColumn(
    "INVOICE DATE NEXT YEAR",
    F.when(F.col("`INVOICE DATE`").isNull(), F.lit(None).cast(DateType()))
     .otherwise(F.add_months(F.col("`INVOICE DATE`"), 12))
)

# --- Replace null/empty EFFECTIVE DATE with INVOICE DATE ---
df = df.withColumn(
    "EFFECTIVE DATE",
    F.coalesce(F.col("`EFFECTIVE DATE`"), F.col("`INVOICE DATE`"))
)

# --- Replace null/empty EXPIRY DATE with INVOICE DATE NEXT YEAR ---
df = df.withColumn(
    "EXPIRY DATE",
    F.coalesce(F.col("`EXPIRY DATE`"), F.col("`INVOICE DATE NEXT YEAR`"))
)

# --- Revenue Country from Source.Name (M code lines 57-71) ---
source_name = F.upper(F.col("`Source.Name`"))
df = df.withColumn(
    "Revenue Country",
    F.when(source_name.contains("CHINA"), F.lit("China"))
     .when(source_name.contains("HONGKONG"), F.lit("Hong Kong"))
     .when(source_name.contains("INDONESIA"), F.lit("Indonesia"))
     .when(source_name.contains("TAIWAN"), F.lit("Taiwan"))
     .when(source_name.contains("KOREA"), F.lit("Korea"))
     .when(source_name.contains("PHILIPPINES"), F.lit("Philippines"))
     .when(source_name.contains("AUSTRALIA"), F.lit("Australia"))
     .when(source_name.contains("NEW ZEALAND"), F.lit("New Zealand"))
     .otherwise(F.lit(None))
)

# --- FINAL DATE = INVOICE DATE ---
df = df.withColumn("FINAL DATE", F.col("`INVOICE DATE`"))

# --- FINAL YEAR ---
df = df.withColumn(
    "FINAL YEAR",
    F.when(F.col("`FINAL DATE`").isNull(), F.lit(None))
     .otherwise(F.year(F.col("`FINAL DATE`")).cast(StringType()))
)

# --- CCY from Revenue Country (M code lines 89-103) ---
df = df.withColumn(
    "CCY",
    F.when(F.col("`Revenue Country`") == "China", F.lit("CNY"))
     .when(F.col("`Revenue Country`") == "Hong Kong", F.lit("HKD"))
     .when(F.col("`Revenue Country`") == "Indonesia", F.lit("IDR"))
     .when(F.col("`Revenue Country`") == "Taiwan", F.lit("TWD"))
     .when(F.col("`Revenue Country`") == "Korea", F.lit("KRW"))
     .when(F.col("`Revenue Country`") == "Philippines", F.lit("PHP"))
     .when(F.col("`Revenue Country`") == "Australia", F.lit("AUD"))
     .when(F.col("`Revenue Country`") == "New Zealand", F.lit("NZD"))
     .otherwise(F.lit(None))
)

# --- CCYYEAR ---
df = df.withColumn(
    "CCYYEAR",
    F.when(
        F.col("CCY").isNull() | F.col("`FINAL YEAR`").isNull(),
        F.lit(None)
    ).otherwise(F.concat(F.col("CCY"), F.lit("-"), F.col("`FINAL YEAR`")))
)

# --- Branch Mapping Join — HK only (Alteryx Tool 304) ---
# Replaces INSURER COUNTRY for HK rows using INS BRANCH → BRANCH lookup
df_branch = spark.table(REF_BRANCH_MAPPING)
df = df.join(
    df_branch.select(
        F.col("BRANCH").alias("_branch_key"),
        F.col("`INSURER COUNTRY`").alias("_branch_insurer_country")
    ),
    F.col("`INS BRANCH`") == F.col("_branch_key"),
    "left"
)

# For HK rows: use branch mapping INSURER COUNTRY; others: keep original
df = df.withColumn(
    "INSURER COUNTRY",
    F.when(
        F.col("`Revenue Country`") == "Hong Kong",
        F.coalesce(F.col("_branch_insurer_country"), F.col("`INSURER COUNTRY`"))
    ).otherwise(F.col("`INSURER COUNTRY`"))
).drop("_branch_key", "_branch_insurer_country")

# --- TRIM + UPPER: RISK DESCRIPTION, INSURER NAME, CCYYEAR (Alteryx Tool 204) ---
df = df.withColumn(
    "RISK DESCRIPTION",
    F.when(F.col("`RISK DESCRIPTION`").isNull(), F.lit(None))
     .otherwise(F.trim(F.upper(F.col("`RISK DESCRIPTION`"))))
)
df = df.withColumn(
    "INSURER NAME",
    F.when(F.col("`INSURER NAME`").isNull(), F.lit(None))
     .otherwise(F.trim(F.upper(F.col("`INSURER NAME`"))))
)
df = df.withColumn(
    "CCYYEAR",
    F.when(F.col("CCYYEAR").isNull(), F.lit(None))
     .otherwise(F.trim(F.upper(F.col("CCYYEAR"))))
)

# --- Replace null RISK DESCRIPTION with "Unknown" ---
df = df.withColumn(
    "RISK DESCRIPTION",
    F.when(F.col("`RISK DESCRIPTION`").isNull(), F.lit("Unknown"))
     .otherwise(F.col("`RISK DESCRIPTION`"))
)

print(f"After transformations: {df.count()} rows")

## Cell 4 — Reference Table Joins

| Join | Left Key | Right Table | Right Key | Columns Brought In |
|------|----------|-------------|-----------|--------------------|
| Currency | `CCYYEAR` | `ref_Chloe_asia_currency_mapping` | `CCYYEAR` | `Value` → `CCYVALUE` |
| Product | `RISK DESCRIPTION` | `ref_Chloe_eglobal_product_mapping` | `SYSTEM PRODUCT ID` | `Sub Product Class`, `GLOBs`, `GLOBS SPLIT P&C` |
| Insurer | `INSURER NAME` | `ref_Chloe_insurer_mapping` | `Insurer` | `MAPPED_INSURER`, `Lloyd's Asia or Lloyd's London` |

In [ ]:
# =============================================================================
# Cell 5: Reference Joins + Derived Columns + Final Select
# Alteryx: Join (61) CCYYEAR, Join (198) Product, Join (199) Insurer,
#          Formula (66), Select (195)
# =============================================================================

# --- 1. Currency mapping join on CCYYEAR (Alteryx Tool 61) ---
df_ccy = spark.table(REF_CURRENCY_MAPPING).select(
    F.trim(F.upper(F.col("CCYYEAR"))).alias("_ccy_key"),
    F.col("Value").alias("CCYVALUE")
)
df = df.join(df_ccy, F.col("CCYYEAR") == F.col("_ccy_key"), "left").drop("_ccy_key")

# --- 2. Product mapping join on RISK DESCRIPTION (Alteryx Tool 198) ---
df_product = spark.table(REF_PRODUCT_MAPPING).select(
    F.trim(F.upper(F.col("`SYSTEM PRODUCT ID`"))).alias("_prod_key"),
    F.col("`Sub Product Class`"),
    F.col("GLOBs"),
    F.col("`GLOBS SPLIT P&C`")
)
df = df.join(df_product, F.col("`RISK DESCRIPTION`") == F.col("_prod_key"), "left").drop("_prod_key")

# --- 3. Insurer mapping join on INSURER NAME (Alteryx Tool 199) ---
df_insurer = spark.table(REF_INSURER_MAPPING).select(
    F.trim(F.upper(F.col("Insurer"))).alias("_ins_key"),
    F.col("MAPPED_INSURER"),
    F.col("`Lloyd's Asia or Lloyd's London`")
)
df = df.join(df_insurer, F.col("`INSURER NAME`") == F.col("_ins_key"), "left").drop("_ins_key")

# --- 4. Derived columns (Alteryx Tool 66) ---

# PREMIUM (USD) = Premium * CCYVALUE  (null-safe, coalesce to 0)
df = df.withColumn(
    "PREMIUM (USD)",
    F.coalesce(
        F.col("Premium").cast(DoubleType()) * F.col("CCYVALUE").cast(DoubleType()),
        F.lit(0.0)
    )
)

# BROKERAGE (USD) = Brokerage * CCYVALUE  (null-safe, coalesce to 0)
df = df.withColumn(
    "BROKERAGE (USD)",
    F.coalesce(
        F.col("Brokerage").cast(DoubleType()) * F.col("CCYVALUE").cast(DoubleType()),
        F.lit(0.0)
    )
)

# SYSTEM ID = COMPANY + BRANCH + CLIENT NUMBER
df = df.withColumn(
    "SYSTEM ID",
    F.when(
        F.col("COMPANY").isNull() | F.col("BRANCH").isNull() | F.col("`CLIENT NUMBER`").isNull(),
        F.lit(None)
    ).otherwise(
        F.concat(F.col("COMPANY"), F.col("BRANCH"), F.col("`CLIENT NUMBER`"))
    )
)

# CLIENTID = if Party ID is null then SYSTEM ID else Party ID
df = df.withColumn(
    "CLIENTID",
    F.coalesce(F.col("`Party ID`"), F.col("`SYSTEM ID`"))
)

# BUSINESS TYPE = "Unknown"
df = df.withColumn("BUSINESS TYPE", F.lit("Unknown"))

# DATA SOURCE = "EglobaL" (confirmed by user)
df = df.withColumn("DATA SOURCE", F.lit("EglobaL"))

# REINSURANCE DESCRIPTION (M code logic — correct version)
df = df.withColumn(
    "REINSURANCE DESCRIPTION",
    F.when(
        F.col("`RISK DESCRIPTION`").isNotNull() &
        F.upper(F.col("`RISK DESCRIPTION`")).contains("REINSURANCE"),
        F.concat(
            F.lit("Reinsurance:Refer to Policy Description"),
            F.coalesce(F.col("`Policy Description`"), F.lit(""))
        )
    ).otherwise(F.lit("null"))
)

# --- 5. Final Select: 28 columns with PascalCase aliases ---
# Following other notebooks: only keep Lloyd's Asia or Lloyd's London (aliased Lloyds)
# Numeric columns coalesced to 0 for null safety
df_final = df.select(
    F.col("`INVOICE NO`").cast(StringType()).alias("InvoicePolicyNumber"),
    F.col("`POLICY DEPT`").cast(StringType()).alias("Department"),
    F.col("`INSURER NAME`").cast(StringType()).alias("InsurerName"),
    F.col("`INSURER COUNTRY`").cast(StringType()).alias("InsurerCountry"),
    F.col("`CLIENT NAME`").cast(StringType()).alias("ClientName"),
    F.col("`RISK DESCRIPTION`").cast(StringType()).alias("SystemProductId"),
    F.col("`INVOICE DATE`").cast(DateType()).alias("InvoiceDate"),
    F.col("`EFFECTIVE DATE`").cast(DateType()).alias("InceptionDate"),
    F.col("`EXPIRY DATE`").cast(DateType()).alias("ExpiryDate"),
    F.col("`ACCOUNT HANDLER`").cast(StringType()).alias("AccountHandler"),
    F.col("`Transaction Type`").cast(StringType()).alias("TransactionType"),
    F.col("`Policy Description`").cast(StringType()).alias("PolicyDescription"),
    F.col("`Party ID`").cast(StringType()).alias("PartyIdWtw"),
    F.col("`DUNS Number`").cast(StringType()).alias("DunsNumber"),
    F.col("`Revenue Country`").cast(StringType()).alias("RevenueCountry"),
    F.col("`FINAL DATE`").cast(DateType()).alias("FinalDate"),
    F.col("`Sub Product Class`").cast(StringType()).alias("SubProductClass"),
    F.col("GLOBs").cast(StringType()).alias("Globs"),
    F.col("`GLOBS SPLIT P&C`").cast(StringType()).alias("GlobsSplitPc"),
    F.col("MAPPED_INSURER").cast(StringType()).alias("InsurerMapping"),
    F.col("`Lloyd's Asia or Lloyd's London`").cast(StringType()).alias("Lloyds"),
    F.coalesce(F.col("`PREMIUM (USD)`"), F.lit(0.0)).cast(DoubleType()).alias("PremiumUsd"),
    F.coalesce(F.col("`BROKERAGE (USD)`"), F.lit(0.0)).cast(DoubleType()).alias("BrokerageUsd"),
    F.col("`SYSTEM ID`").cast(StringType()).alias("SystemId"),
    F.col("CLIENTID").cast(StringType()).alias("ClientIdWtw"),
    F.col("`BUSINESS TYPE`").cast(StringType()).alias("BusinessType"),
    F.col("`DATA SOURCE`").cast(StringType()).alias("DataSource"),
    F.col("`REINSURANCE DESCRIPTION`").cast(StringType()).alias("ReinsuranceDescription")
)

print(f"Final columns: {len(df_final.columns)}")
df_final.printSchema()

In [ ]:
# =============================================================================
# Cell 6: Write to Silver
# =============================================================================

# --- Standardize column order across all silver notebooks ---
STANDARD_COLUMNS = [
    "AccountHandler", "BrokerageUsd", "BusinessType", "ClientIdWtw", "ClientName",
    "DataSource", "Department", "DunsNumber", "ExpiryDate", "FinalDate",
    "Globs", "GlobsSplitPc", "InceptionDate", "InsurerCountry", "InsurerMapping",
    "InsurerName", "InvoiceDate", "InvoicePolicyNumber", "Lloyds", "PartyIdWtw",
    "PolicyDescription", "PremiumUsd", "ReinsuranceDescription", "RevenueCountry",
    "SubProductClass", "SystemId", "SystemProductId", "TransactionType"
]
df_final = df_final.select(*STANDARD_COLUMNS)

print(f"Writing to {TARGET_TABLE}...")
df_final.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(TARGET_TABLE)

print(f"Success. Rows written: {spark.table(TARGET_TABLE).count()}")
print(f"Columns: {len(spark.table(TARGET_TABLE).columns)}")
display(spark.table(TARGET_TABLE).limit(5))